# SalesPulse: Data Preparation & Integration
## Notebook 2 of 4

This notebook focuses on:
1. Advanced cleaning (outlier clipping & categorical standardization)
2. Standardizing schemas for multi-source integration
3. Applying the Feature Engineering module
4. Exporting processed data for forecasting

In [6]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.append('../src')
from feature_engineering import FeatureEngineer

print("✓ Environment initialized")

✓ Environment initialized


## 1. Clean and Standardize Online Retail
Standardizing names and calculating sales.

In [7]:
import os
import pandas as pd

retail_path = '../data/raw/Online Retail.csv'
retail_std = pd.DataFrame() 

if os.path.exists(retail_path):
    df_retail = pd.read_csv(retail_path, encoding='utf-8-sig')
    df_retail.columns = df_retail.columns.str.strip()
    
    retail_rename_map = {
        'InvoiceNo': 'order_id',
        'StockCode': 'product_id',
        'Quantity': 'quantity',
        'InvoiceDate': 'order_date',
        'UnitPrice': 'unit_price',
        'CustomerID': 'customer_id',
        'Country': 'region'
    }
    
    retail_std = df_retail.rename(columns=retail_rename_map)
    
    retail_std = retail_std.dropna(subset=['customer_id'])
    retail_std = retail_std[(retail_std['quantity'] > 0) & (retail_std['unit_price'] > 0)]
    
    retail_std['sales'] = retail_std['quantity'] * retail_std['unit_price']
    retail_std['order_date'] = pd.to_datetime(retail_std['order_date'])
    
    target_cols = ["order_id", "order_date", "customer_id", "product_id", "quantity", "sales", "region", "unit_price"]
    retail_std = retail_std[target_cols]
    
    print(f"Retail Cleaned: {retail_std.shape}")

Retail Cleaned: (397884, 8)


## 2. Clean and Standardize Superstore
Aligning formatting with the Retail dataset.

In [8]:
df_superstore = pd.read_csv('../data/raw/superstore_final_dataset.csv', encoding='ISO-8859-1')
df_superstore.columns = df_superstore.columns.str.strip()

df_superstore['Order_Date'] = pd.to_datetime(df_superstore['Order_Date'], dayfirst=True)

super_std = df_superstore.rename(columns={
    'Order_ID': 'order_id',
    'Order_Date': 'order_date',
    'Customer_ID': 'customer_id',
    'Product_ID': 'product_id',
    'Sales': 'sales',
    'Region': 'region'
})

super_std['quantity'] = 1 
super_std['unit_price'] = super_std['sales']

super_std = super_std[["order_id", "order_date", "customer_id", "product_id", "quantity", "sales", "region", "unit_price"]]
print(f"Superstore Cleaned: {super_std.shape}")

Superstore Cleaned: (9800, 8)


## 3. Merging and Feature Engineering

In [9]:
df_merged = pd.concat([super_std, retail_std], ignore_index=True)

engineer = FeatureEngineer(df_merged)
df_final = engineer.run_all_engineering()
views = engineer.create_aggregated_views()

print(f"Final Integrated Shape: {df_final.shape}")


=== Running Feature Engineering ===
✓ Created time features: year, month, quarter, day_of_week, etc.
✓ Created transaction features: effective_price, revenue_per_item, etc.
✓ Created customer features: RFM metrics, customer segments, etc.
✓ Created product features: popularity, total revenue, etc.

Feature engineering complete: 39 total features
✓ Created 3 aggregated views
Final Integrated Shape: (407684, 39)


## 4. Export to Processed

In [10]:
os.makedirs('../data/processed', exist_ok=True)
df_final.to_csv('../data/processed/cleaned_sales.csv', index=False)
views['daily_sales'].to_csv('../data/processed/daily_sales.csv', index=False)
views['monthly_sales'].to_csv('../data/processed/monthly_sales.csv', index=False)

print("✓ Processed data saved for analysis")

✓ Processed data saved for analysis
